<a href="https://colab.research.google.com/github/amalrik1/develop_ml_course/blob/main/practice3/ecg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import optuna

data = pd.read_csv('/content/drive/MyDrive/ecg.txt')
data

,SAMPLE,IsTestSet,CLASS,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,VAR8,...,VAR133,VAR134,VAR135,VAR136,VAR137,VAR138,VAR139,VAR140,VAR141,OBJECT
0,train,False,1,-0.112522,-2.827204,-3.773897,-4.349751,-4.376041,-3.474986,-2.181408,...,0.792168,0.933541,0.796958,0.578621,0.257740,0.228077,0.123431,0.925286,0.193137,obj1
1,train,False,1,0.800232,-0.874252,-2.384761,-3.973292,-4.338224,-3.802422,-2.534510,...,1.148884,0.958434,1.059025,1.371682,1.277392,0.960304,0.971020,1.614392,1.421456,obj5
2,train,False,1,-0.297161,-2.766635,-4.102185,-4.589669,-4.219357,-3.650443,-2.300518,...,0.581779,0.684406,0.911651,0.979483,1.053458,0.974787,1.110407,1.288165,-0.823386,obj7
3,train,False,1,0.446769,-1.507397,-3.187468,-4.507462,-4.604201,-3.636115,-2.311604,...,1.001306,1.292059,1.378667,1.014765,0.820793,1.034388,1.258433,0.961215,-0.999476,obj8
4,train,False,1,-0.832281,-1.700368,-2.257301,-2.853671,-2.853301,-2.701487,-2.285726,...,2.110504,2.203668,2.227544,2.171733,2.045938,2.126372,2.126852,1.679299,0.965814,obj10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3228,test,True,-1,-0.204939,-1.068641,-1.765908,-2.611745,-2.928615,-3.040122,-2.644041,...,-2.207918,-2.611683,-2.936433,-3.428299,-3.534629,-3.161665,-1.986092,-1.122876,-0.068618,obj3221
3229,test,True,-1,0.471892,-0.418815,-1.480933,-2.426561,-3.183361,-3.505937,-3.434058,...,1.447986,2.259450,2.616897,2.134736,1.694801,1.496465,1.537012,1.111047,0.661459,obj3222
3230,test,True,-1,-0.845808,-0.837969,-1.252438,-2.451468,-2.874559,-3.153296,-3.699509,...,0.929398,0.642008,0.454321,0.269527,0.906880,1.335777,1.139420,0.635677,0.562999,obj3224
3231,test,True,-1,-0.415214,-1.383605,-2.028370,-2.663996,-3.308097,-3.618700,-3.515099,...,1.507329,1.712179,2.417778,2.380480,1.671464,1.069622,0.814390,0.659216,0.253140,obj3227


In [2]:
X_train = data.loc[data['SAMPLE'] == 'train']
X_valid = data.loc[data['SAMPLE'] == 'valid']
X_valid.reset_index(drop=True, inplace=True)
X_test = data.loc[data['SAMPLE'] == 'test']
X_test.reset_index(drop=True, inplace=True)

In [3]:
Y_train = X_train['CLASS']
X_train = X_train.drop(['SAMPLE','IsTestSet','CLASS','OBJECT'], axis = 1)

In [5]:
Y_valid = X_valid['CLASS']
Y_valid.reset_index(drop=True, inplace=True)
Y_valid

0      1
1      1
2      1
3      1
4      1
      ..
590   -1
591   -1
592   -1
593   -1
594   -1
Name: CLASS, Length: 595, dtype: int64

In [6]:
X_valid = X_valid.drop(['SAMPLE','IsTestSet','CLASS','OBJECT'], axis = 1)
X_valid

,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,VAR8,VAR9,VAR10,VAR11,...,VAR132,VAR133,VAR134,VAR135,VAR136,VAR137,VAR138,VAR139,VAR140,VAR141
0,0.087631,-1.753490,-3.304473,-4.704657,-4.686415,-3.611817,-2.267268,-1.570893,-1.417790,-0.500788,...,1.575754,1.709046,2.142926,2.393528,1.925550,1.027624,0.573453,0.192971,-0.648683,-2.441068
1,2.744603,-0.101924,-2.851681,-3.974909,-3.995437,-3.719663,-2.680970,-1.441639,-1.166864,-0.657178,...,0.503609,0.451441,0.802779,0.883922,0.895320,0.840126,1.305181,2.114687,1.590270,-1.921928
2,2.402869,2.036719,0.340902,-1.286436,-3.301455,-4.224569,-3.398307,-2.079302,-1.597905,-1.527974,...,0.118138,0.416087,1.114983,0.899164,1.154525,1.987328,2.428732,2.613987,1.962819,1.769198
3,-3.527899,-5.117621,-4.634885,-4.226482,-3.100461,-1.897723,-1.306379,-0.952464,-0.228996,-0.096763,...,0.685109,0.756051,0.967958,0.975465,0.914873,0.640022,0.511706,0.523466,0.805130,-0.912395
4,-0.869182,-2.206586,-2.989571,-3.510035,-3.289364,-3.007315,-2.240611,-1.711585,-1.805550,-1.418332,...,1.166905,1.296914,1.288558,1.533431,1.509486,1.390213,1.365502,1.322504,1.852126,1.541934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,-0.081543,-0.885183,-1.863528,-2.491558,-3.090885,-3.721665,-3.646080,-3.208407,-2.631355,-1.850419,...,1.552767,1.410982,1.292504,1.749073,2.411167,2.266412,1.369975,0.864262,0.799865,0.654732
591,-0.669841,-1.460590,-2.347707,-2.707371,-2.946708,-3.419234,-3.549447,-3.452586,-2.927853,-2.364581,...,1.569806,1.370021,1.183312,1.242766,1.783490,1.913804,1.376940,0.747530,0.422273,0.431464
592,0.089044,-0.098339,-0.648722,-1.499857,-1.878173,-2.321745,-2.858452,-3.132297,-2.992125,-2.572737,...,1.679414,1.564307,1.593124,1.620218,1.870083,2.297747,2.790642,2.664542,2.335217,2.160673
593,-0.097026,-1.188811,-2.059970,-2.987246,-3.459390,-3.472208,-3.263205,-2.761881,-1.959433,-1.308527,...,1.500864,1.877755,2.422262,2.536718,1.847567,1.398425,1.197775,1.309180,0.848214,0.418162


In [7]:
Y_test = X_test['CLASS']
Y_test.reset_index(drop=True, inplace=True)
X_test = X_test.drop(['SAMPLE','IsTestSet','CLASS','OBJECT'], axis = 1)

In [8]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
scl = StandardScaler()
columns = X_train.columns
columns

Index(['VAR2', 'VAR3', 'VAR4', 'VAR5', 'VAR6', 'VAR7', 'VAR8', 'VAR9', 'VAR10',
       'VAR11',
       ...
       'VAR132', 'VAR133', 'VAR134', 'VAR135', 'VAR136', 'VAR137', 'VAR138',
       'VAR139', 'VAR140', 'VAR141'],
      dtype='object', length=140)

In [9]:
X_train_scaled = pd.DataFrame(scl.fit_transform(X_train[columns]))
X_train_scaled

,0,1,2,3,4,5,6,7,8,9,...,130,131,132,133,134,135,136,137,138,139
0,0.208635,-0.431160,-0.478049,-0.563173,-0.947402,-0.325130,0.144872,-0.413491,-0.135167,0.321842,...,-1.029686,-0.306603,-0.212951,-0.320740,-0.361648,-0.373007,-0.279127,-0.356727,0.350662,0.586220
1,0.929305,0.933741,0.906111,-0.032465,-0.879164,-0.782799,-0.368708,-0.337332,-0.838422,-0.225530,...,0.275294,0.311969,-0.168824,0.050238,0.537285,0.653387,0.443267,0.474962,1.020346,1.562014
2,0.062852,-0.388829,-0.805161,-0.901394,-0.664676,-0.570373,-0.028370,0.732000,0.242838,0.295976,...,-0.326209,-0.671434,-0.654582,-0.158383,0.092728,0.427973,0.457556,0.611735,0.703314,-0.221321
3,0.650226,0.491241,0.106279,-0.785504,-1.359102,-0.550346,-0.044495,0.068323,-0.364034,-0.058796,...,0.472107,0.056057,0.422578,0.502717,0.132720,0.193769,0.516356,0.756984,0.385578,-0.361209
4,-0.359656,0.356376,1.033114,1.545905,1.800287,0.756020,-0.006856,0.160543,-0.168088,-0.886167,...,1.993719,1.979489,2.038545,1.704374,1.444142,1.427014,1.593676,1.609112,1.083423,1.200045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2038,-0.995275,-0.721969,-0.667842,-0.587146,0.042660,0.246798,0.627489,1.201363,0.262388,0.493305,...,0.719986,0.289208,-0.146792,-0.712742,-0.941217,-1.144838,-1.805847,-2.243304,-1.843691,-1.073917
2039,-1.039998,-0.774646,-0.488990,-0.173892,-0.023913,-0.146434,-0.060071,-0.046760,-0.552030,-0.471828,...,0.618592,0.459731,0.275851,0.202545,0.006975,0.009189,0.427623,0.333779,-0.725536,-1.095908
2040,-1.352633,-0.539902,-0.554456,0.125137,1.463449,2.600460,2.414448,3.808801,4.534172,3.869574,...,-1.416816,-1.038957,-0.763288,0.022359,0.134571,-0.330823,-1.463047,-2.227697,-2.495937,-2.347927
2041,-1.379518,-0.724789,-0.441602,-0.122745,0.157284,0.239766,0.752782,0.498962,0.322095,0.593005,...,-0.312982,-0.835888,-1.083029,-1.211751,-0.879278,-0.858805,-0.798722,-0.668563,-1.522186,-1.885961


In [10]:
X_valid_scaled = pd.DataFrame(scl.transform(X_valid[columns]))

In [11]:
X_valid_scaled

,0,1,2,3,4,5,6,7,8,9,...,130,131,132,133,134,135,136,137,138,139
0,0.366666,0.319249,-0.010307,-1.063496,-1.507453,-0.516383,0.019991,0.126943,-0.477192,0.275591,...,1.261239,1.283331,1.930871,1.939337,1.165093,0.401967,0.061611,-0.288491,-1.178945,-1.506431
1,2.464494,1.473515,0.440864,-0.034744,-0.260627,-0.667125,-0.581732,0.409300,0.035896,-0.034896,...,-0.474096,-0.897450,-0.444748,-0.197635,-0.002671,0.213230,0.783513,1.597175,0.996904,-1.094018
2,2.194677,2.968194,3.622012,3.755293,0.991621,-1.372850,-1.625087,-0.983684,-0.845487,-1.763720,...,-1.098004,-0.958755,0.108683,-0.176059,0.291138,1.368017,1.891976,2.087109,1.358953,1.838265
3,-2.487997,-2.031913,-1.335954,-0.389396,1.354302,1.879470,1.417588,1.477910,1.953622,1.077719,...,-0.180327,-0.369232,-0.151943,-0.068048,0.019493,0.011803,0.000693,0.035804,0.233893,-0.292031
4,-0.388791,0.002584,0.303467,0.620605,1.013438,0.328552,0.058763,-0.180400,-1.270073,-1.546044,...,0.599490,0.568665,0.416371,0.721799,0.693486,0.766953,0.843024,0.819852,1.251379,1.657723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,0.233094,0.926102,1.425478,2.056388,1.371582,-0.669923,-1.985469,-3.450228,-2.958657,-2.403885,...,1.224032,0.766467,0.423367,1.027058,1.715539,1.648946,0.847436,0.370207,0.228776,0.952917
591,-0.231400,0.523955,0.943033,1.752150,1.631740,-0.247203,-1.844917,-3.983639,-3.564929,-3.424671,...,1.251612,0.695437,0.229807,0.310339,1.004069,1.294007,0.854308,0.255664,-0.138174,0.775550
592,0.367782,1.476020,2.635932,3.454425,3.559843,1.286799,-0.839876,-3.283965,-3.696349,-3.837930,...,1.429018,1.032343,0.956262,0.844653,1.102222,1.680489,2.249027,2.136715,1.720855,2.149258
593,0.220869,0.713899,1.229738,1.357600,0.706637,-0.321246,-1.428584,-2.474786,-1.584728,-1.328045,...,1.140024,1.575885,2.426037,2.142035,1.076700,0.775220,0.677549,0.806778,0.275763,0.764982


In [25]:
X_test_scaled = pd.DataFrame(scl.transform(X_test[columns]))

In [17]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix,fbeta_score,f1_score


In [14]:
def run(trial: optuna.Trial):
  params = {
      'n_neighbors': trial.suggest_int('n_neighbors', 15, 25, step=1),
      'contamination': trial.suggest_float('contamination', 0.01, 0.06, step = 0.01)
  }
  lof =  LocalOutlierFactor(**params, novelty = True)

  lof.fit(X_train_scaled)

  score = fbeta_score(Y_valid, lof.predict(X_valid_scaled),pos_label=-1, beta = 4)
  return score

In [15]:
lof =  LocalOutlierFactor(novelty = True)
lof.fit(X_train_scaled)


LocalOutlierFactor(novelty=True)

In [19]:
import datetime
now = datetime.datetime.now()
study = optuna.create_study(direction='maximize')
study.optimize(run, n_trials = 65)
print(datetime.datetime.now() - now)

[I 2023-10-03 09:56:13,938] A new study created in memory with name: no-name-7492f6d3-5c2b-4919-90ec-0e74ac4b3b19
[I 2023-10-03 09:56:14,126] Trial 0 finished with value: 0.9623836126629421 and parameters: {'n_neighbors': 16, 'contamination': 0.05}. Best is trial 0 with value: 0.9623836126629421.
[I 2023-10-03 09:56:14,316] Trial 1 finished with value: 0.8832831325301206 and parameters: {'n_neighbors': 18, 'contamination': 0.02}. Best is trial 0 with value: 0.9623836126629421.
[I 2023-10-03 09:56:14,496] Trial 2 finished with value: 0.8950733358405417 and parameters: {'n_neighbors': 21, 'contamination': 0.02}. Best is trial 0 with value: 0.9623836126629421.
[I 2023-10-03 09:56:14,687] Trial 3 finished with value: 0.9472700074794316 and parameters: {'n_neighbors': 23, 'contamination': 0.03}. Best is trial 0 with value: 0.9623836126629421.
[I 2023-10-03 09:56:14,868] Trial 4 finished with value: 0.9631010063361909 and parameters: {'n_neighbors': 18, 'contamination': 0.05}. Best is trial 

0:00:10.385021


In [27]:
lof2 = LocalOutlierFactor(n_neighbors=18,contamination=0.05,novelty = True)
lof2.fit(X_train_scaled)
Y_lof = lof.predict(X_valid_scaled)
score = f1_score(Y_valid, Y_lof,pos_label=-1)
tn, fp, fn, tp = confusion_matrix(Y_valid, Y_lof).ravel()
print(score)
print(study.best_params)
print(study.best_value)
print(tn, fp, fn, tp )

0.9212121212121213
{'n_neighbors': 18, 'contamination': 0.05}
0.9631010063361909
152 5 21 417


In [21]:
X_valid_scaled

,0,1,2,3,4,5,6,7,8,9,...,130,131,132,133,134,135,136,137,138,139
0,0.366666,0.319249,-0.010307,-1.063496,-1.507453,-0.516383,0.019991,0.126943,-0.477192,0.275591,...,1.261239,1.283331,1.930871,1.939337,1.165093,0.401967,0.061611,-0.288491,-1.178945,-1.506431
1,2.464494,1.473515,0.440864,-0.034744,-0.260627,-0.667125,-0.581732,0.409300,0.035896,-0.034896,...,-0.474096,-0.897450,-0.444748,-0.197635,-0.002671,0.213230,0.783513,1.597175,0.996904,-1.094018
2,2.194677,2.968194,3.622012,3.755293,0.991621,-1.372850,-1.625087,-0.983684,-0.845487,-1.763720,...,-1.098004,-0.958755,0.108683,-0.176059,0.291138,1.368017,1.891976,2.087109,1.358953,1.838265
3,-2.487997,-2.031913,-1.335954,-0.389396,1.354302,1.879470,1.417588,1.477910,1.953622,1.077719,...,-0.180327,-0.369232,-0.151943,-0.068048,0.019493,0.011803,0.000693,0.035804,0.233893,-0.292031
4,-0.388791,0.002584,0.303467,0.620605,1.013438,0.328552,0.058763,-0.180400,-1.270073,-1.546044,...,0.599490,0.568665,0.416371,0.721799,0.693486,0.766953,0.843024,0.819852,1.251379,1.657723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,0.233094,0.926102,1.425478,2.056388,1.371582,-0.669923,-1.985469,-3.450228,-2.958657,-2.403885,...,1.224032,0.766467,0.423367,1.027058,1.715539,1.648946,0.847436,0.370207,0.228776,0.952917
591,-0.231400,0.523955,0.943033,1.752150,1.631740,-0.247203,-1.844917,-3.983639,-3.564929,-3.424671,...,1.251612,0.695437,0.229807,0.310339,1.004069,1.294007,0.854308,0.255664,-0.138174,0.775550
592,0.367782,1.476020,2.635932,3.454425,3.559843,1.286799,-0.839876,-3.283965,-3.696349,-3.837930,...,1.429018,1.032343,0.956262,0.844653,1.102222,1.680489,2.249027,2.136715,1.720855,2.149258
593,0.220869,0.713899,1.229738,1.357600,0.706637,-0.321246,-1.428584,-2.474786,-1.584728,-1.328045,...,1.140024,1.575885,2.426037,2.142035,1.076700,0.775220,0.677549,0.806778,0.275763,0.764982
